# 준비

- kiwi 설치

In [1]:
!pip install kiwipiepy

- 데이터 다운로드

In [2]:
!wget -c https://github.com/e9t/nsmc/raw/master/ratings_train.txt

--2021-04-02 10:34:13--  https://github.com/e9t/nsmc/raw/master/ratings_train.txt
Resolving github.com (github.com)... 15.164.81.167
Connecting to github.com (github.com)|15.164.81.167|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt [following]
--2021-04-02 10:34:13--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: 'ratings_train.txt'

     0K .......... .......... .......... .......... ..........  0% 1.15M 12s
    50K .......... .......... .......... .......... ..........  0% 4.68M 7s
   100K .......... .......... .......... .......... ..........  1% 1.3

  7100K .......... .......... .......... .......... .......... 50% 34.6M 0s
  7150K .......... .......... .......... .......... .......... 50% 30.6M 0s
  7200K .......... .......... .......... .......... .......... 50% 40.6M 0s
  7250K .......... .......... .......... .......... .......... 51% 37.6M 0s
  7300K .......... .......... .......... .......... .......... 51% 29.4M 0s
  7350K .......... .......... .......... .......... .......... 51% 31.8M 0s
  7400K .......... .......... .......... .......... .......... 52% 25.2M 0s
  7450K .......... .......... .......... .......... .......... 52% 35.5M 0s
  7500K .......... .......... .......... .......... .......... 52% 27.6M 0s
  7550K .......... .......... .......... .......... .......... 53% 15.9M 0s
  7600K .......... .......... .......... .......... .......... 53% 37.8M 0s
  7650K .......... .......... .......... .......... .......... 53% 3.71M 0s
  7700K .......... .......... .......... .......... .......... 54% 27.2M 0s
  7750K ....

# 데이터 열기

In [3]:
import pandas as pd

In [4]:
nsmc = pd.read_csv('../data/ratings_train.txt', sep='\t')

In [5]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 형태소 분석

In [6]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

0

In [8]:
text = nsmc.document[0]

In [9]:
result = kiwi.analyze(text)

In [12]:
for token, pos, start, end in result[0][0]:
    if pos[0] in 'NV': # if pos in ("NNG", "VV", "VA"): 포괄적이지 못함
        print(f'{token}/{pos}') # print(token, pos)

더빙/NNG
짜증/NNG
나/VV
목소리/NNG


In [13]:
def extract_keywords(text):
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if pos[0] in 'NV':
            yield f'{token}/{pos}'

In [14]:
list(extract_keywords(nsmc.document[0]))

['더빙/NNG', '짜증/NNG', '나/VV', '목소리/NNG']

# 문서 단어 행렬

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [17]:
nsmc.shape

(150000, 3)

In [18]:
cv = CountVectorizer(max_features=1000, tokenizer=extract_keywords)
dtm = cv.fit_transform(nsmc.loc[0:1999, 'document'])

In [19]:
trans = TfidfTransformer()
dtm2 = trans.fit_transform(dtm)

In [ ]:
dtm.shape

In [20]:
words = cv.get_feature_names()

# 저장

In [21]:
import joblib
joblib.dump({'words': words, 'dtm': dtm, 'dtm2': dtm2}, 'nsmc.pkl')

['nsmc.pkl']